In [ ]:
# dependencies
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# importing csv's
scotch_file = 'Resources/scotch_score.csv'
scotch_region_df = pd.read_csv(scotch_file)
scotch_region_df.head()

In [ ]:
scotch_region_df['NAME'].nunique()

In [ ]:
whisky_file = 'Resources/whisky.csv'
whisky_df = pd.read_csv(whisky_file)
whisky_df.head()

In [ ]:
whisky_df.Distillery.nunique()